## Importando dependências

In [ ]:
!python -m pip install scikit-learn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Importando dados

In [ ]:
df_main = pd.read_csv('data/IM_230626_semNP.csv')

## Carteira Industrial

A métrica utilizada será a taxa de inadimplência, calculada pela carteira de direitos de aquisição inadimplentes dividido pelo patrimônio líquido

In [ ]:
#Cálculo da taxa de inadimplência em df_main, onde a taxa é definida pela divisão da coluna "Carteira_Direitos_Aquisicao_Inadimplentes" pela coluna "Patrimonio_Liquido". 
df_main = df_main.dropna(subset=["Patrimonio_Liquido"])
inadimplentes = df_main["Carteira_Direitos_Aquisicao_Inadimplentes"]
patrimonio_liquido = df_main["Patrimonio_Liquido"]
taxa_inadimplencia = inadimplentes / patrimonio_liquido

#Adição da taxa como uma nova coluna no DataFrame
df_main = df_main.assign(taxa_inadimplencia_series=taxa_inadimplencia)

In [ ]:
fundos = df_main[
    (df_main['taxa_inadimplencia_series'] != 0) &
    df_main['taxa_inadimplencia_series'].notna()
]

list(fundos['Nome_Fundo'].unique())

Selecionou-se os fundos cuja taxa de inadimplência fosse válida, ou seja, apenas aqueles que a taxa de inadimplência fosse diferente de 0 e NaN

In [ ]:
#Escolha das carteiras que serão utilizadas
carteiras = [
    'Carteira',
    'Carteira_Industrial',
    'Carteira_Mercado_Imobiliario',
    'Carteira_Comercial_Total',
    'Carteira_Comercial',
    'Carteira_Comercial_Varejo',
    'Carteira_Arrendamento_Mercantil',
    'Carteira_Servicos_Total',
    'Carteira_Servicos',
    'Carteira_Servicos_Publicos',
    'Carteira_Servicos_Educacionais',
    'Carteira_Entretenimento',
    'Carteira_Agronegocio',
    'Carteira_Financeiro',
    'Carteira_Credito_Pessoal_Consignado',
    'Carteira_Credito_Corporativo',
    'Carteira_Middle_Market',
    'Carteira_Veiculos',
    'Carteira_Imobiliaria_Empresarial',
    'Carteira_Imobiliaria_Residencial',
    'Carteira_Outros_Financeiro',
    'Carteira_Cartao_Credito',
    'Carteira_Factoring',
    'Carteira_Factoring_Pessoal',
    'Carteira_Factoring_Corporativo',
    'Carteira_Setor_Publico',
    'Carteira_Precatorios',
    'Carteira_Creditos_Tributarios',
    'Carteira_Royalties',
    'Carteira_Outros_Setor_Publico',
    'Carteira_Acoes_Judiciais',
    'Carteira_Propriedade_Intelectual',
]

#Cálculo da soma das carteiras agrupadas pela taxa de inadimplência e 
# plot de um gráfico de barras para mostrar as taxas totais de inadimplência por carteira
soma_carteiras = df_main.groupby('taxa_inadimplencia_series')[carteiras].sum()
plt.figure(figsize=(14, 8))
ax = sns.barplot(data=soma_carteiras, palette='magma')
plt.title('Taxas de Inadimplência por Carteira')
plt.ylabel('Taxa total de inadimplência')
plt.xlabel('Carteira')
plt.xticks(rotation=45, ha='right')

plt.grid(True)
plt.tight_layout()

plt.show()

O gráfico foi plotado com todos os tipos de carteira, avaliando a taxa de inadimplência de cada segmento.

In [ ]:
soma_carteiras.head(5)

## K-means

O K-means é um algoritmo de agrupamento de dados que divide um conjunto de pontos em grupos (clusters) com base em suas características similares, buscando minimizar a variância dentro de cada grupo.

In [ ]:
from sklearn.cluster import KMeans

#Análise de cluster (K-means) com três clusters
carteira_industrial = 'Carteira_Industrial'

fundos = df_main[df_main[carteira_industrial].notna()]

X = fundos[[carteira_industrial]]

kmeans = KMeans(n_clusters=3)
fundos['grupo'] = kmeans.fit_predict(X)

#Criação de gráfico de dispersão para visualizar os grupos resultantes
cores = ['#FFA500', '#00CED1', '#00008B']
fig, ax = plt.subplots(figsize=(12, 8))

for grupo in range(3):
    grupo_df = fundos[fundos['grupo'] == grupo]
    ax.scatter(grupo_df.index, grupo_df[carteira_industrial],
               color=cores[grupo], label=f'Cluster {grupo}', s=100, alpha=0.7)

ax.set_title(f'{carteira_industrial}', fontsize=16)
ax.set_xlabel('.', fontsize=14)
ax.set_ylabel('Inadimplência', fontsize=14)
ax.set_xticks([])
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

ax.set_facecolor('white')

plt.tight_layout()
plt.show()

In [ ]:
#Filtrar o DataFrame "fundos" para selecionar cluster 1 e classificar os dados com base na coluna "taxa_inadimplencia_series" em ordem decrescente
cluster1 = fundos[fundos['grupo'] == 1]

cluster1 = cluster1.sort_values(by='taxa_inadimplencia_series', ascending=False)

fundos_inadimplencia = cluster1.head(10)

#Imprime informações relevantes sobre os fundos
for index, row in fundos_inadimplencia.iterrows():
    print(f'Nome do Fundo: {row["Nome_Fundo"]}')
    print(f'CNPJ do Fundo: {row["CNPJ"]}')
    print(f'Taxa de Inadimplência: {row["taxa_inadimplencia_series"]}')
    print(f'Data de competência: {row["Data_Competencia"]}')
    print('-' * 30)

Mostra as maiores taxas de inadimplência do cluster 1, ou seja do cluster que agrupa as maiores taxas de inadimplência dos dados.

## DB Scan Industrial e Agronegócio

## Importando dependências

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

## Importando dados

In [ ]:
google_id = '1teoxduT3bJAkrD0k1YIPn-mbVI83oQox'
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id=google_id,
                                    dest_path = './dados_cvm.csv',
                                    showsize = True)

df_main = pd.read_csv('dados_cvm.csv')

google_id = '1J0fF-82tDg70wjaNg2Q8OGxPikA7phqp'
gdd.download_file_from_google_drive(file_id=google_id,
                                    dest_path = './dados_classes.csv',
                                    showsize = True)
df_classes = pd.read_csv('dados_classes.csv')

In [ ]:
# Eliminar as linhas com valores ausentes na coluna "Patrimonio_Liquido" e armazenar o DataFrame limpo em df_cleaned.
df_cleaned = df_main.dropna(subset=["Patrimonio_Liquido"])

# Extrair a coluna "Carteira_Direitos_Aquisicao_Inadimplentes" do DataFrame limpo.
inadimplentes = df_cleaned["Carteira_Direitos_Aquisicao_Inadimplentes"]

# Extrair a coluna "Patrimonio_Liquido" do DataFrame limpo.
patrimonio_liquido = df_cleaned["Patrimonio_Liquido"]

# Calcular a taxa de inadimplência dividindo o número de inadimplentes pelo patrimonio liquido.
taxa_inadimplencia = inadimplentes / patrimonio_liquido

# Apresentar a taxa de inadimplência calculada.
taxa_inadimplencia

# Adicionar uma nova coluna denominada "taxa_inadimplencia_series" ao DataFrame original df_informe_mensal
# e atribuir-lhe os valores da taxa de inadimplência calculada.
df_main = df_main.assign(taxa_inadimplencia_series=taxa_inadimplencia)

In [ ]:
#Criação do Dataframe 
fundos_inadimplentes = df_main[
    (df_main['taxa_inadimplencia_series'] != 0) &
    df_main['taxa_inadimplencia_series'].notna()
]

list(fundos_inadimplentes['Nome_Fundo'].unique())

Selecionou-se as taxas de inadimplência que fosse válida, ou seja, apenas taxas diferentes de 0 e NaN. Em seguida, lista-se os fundos com taxas de inadimplência não nulas e diferentes de zero. 

In [ ]:
#Escolha das carteiras que serão utilizadas
carteiras = [
    'Carteira_Industrial',
    'Carteira_Agronegocio',
]

#Calcular a soma das taxas de inadimplência agrupadas por carteira
#Cria um gráfico de barras para mostrar as somas das taxas de inadimplência por carteira
summed_data = df_main.groupby('taxa_inadimplencia_series')[carteiras].sum()
plt.figure(figsize=(14, 8))
ax = sns.barplot(data=summed_data, palette='viridis')
plt.title('Sum of Default Rates by Portfolio')
plt.xlabel('Total Default Rate')
plt.ylabel('Portfolio')
plt.xticks(rotation=45, ha='right')

plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
#Acessar a coluna extraindo os valores armazenados 
df_main["Carteira_Industrial"]

In [ ]:
#Retorna uma séria de contagem, mostrando quantas vezes cada valor aparece
df_main["Carteira_Industrial"].value_counts()

In [ ]:
#Retorna uma séria de contagem, mostrando quantas vezes cada valor aparece
df_main["Carteira_Agronegocio"].value_counts()

In [ ]:
#Exibe as cinco primeiras linhas de "summed_data"
#Atribui "summed_data" à variável "X" para ser usada posteriormente no código
summed_data.head(5)
X = summed_data

## Carteira Industrial

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# Seleciona as colunas relevantes para o DBSCAN
X = df_main[['Carteira_Industrial', 'Carteira_Agronegocio']]

# Remove os valores NaN do DataFrame
X.dropna(inplace=True)

# Amostragem aleatória de 10% dos dados
sampled_data = X.sample(frac=0.1, random_state=42)

# Aplica o StandardScaler aos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(sampled_data)

# Cria clusters usando DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Ajuste os hiperparâmetros conforme necessário
labels = dbscan.fit_predict(X_scaled)

# Adiciona os rótulos dos clusters ao DataFrame original
sampled_data['Cluster_Labels'] = labels

# Visualiza os clusters
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels, cmap='viridis')
plt.xlabel('Feature 1 (Carteira_Industrial)')
plt.ylabel('Feature 2 (Carteira_Agronegocio)')
plt.title('DBSCAN Clusters')
plt.show()

In [ ]:
sampled_data

In [ ]:
#Retorna uma séria de contagem, mostrando quantas vezes cada valor aparece
sampled_data["Cluster_Labels"].value_counts()

In [ ]:
#Função que recebe argumento "n_cluster" e filtra as linhas do DataFrame
def selecionar_cluster(n_cluster):
  df_cluster = sampled_data.loc[sampled_data["Cluster_Labels"]==n_cluster]
  return df_cluster

In [ ]:
#Adicionar o número de clusters para verificação
selecionar_cluster(0) 

In [ ]:
#Retorna uma séria de contagem, mostrando quantas vezes cada valor aparece
sampled_data["Cluster_Labels"].value_counts()

#Cria gráfico de barras que mostra a distribuição dos clusters 
sampled_data.Cluster_Labels.value_counts().sort_values().plot(kind='bar', color=['green', 'lightgreen', 'yellow', 'purple', 'blue'])
plt.title('Cluster')

In [ ]:
#Cria novo DataFrame retendo apenas as linhas que não pertencem ao cluster 0
df_cluster_s_0 = sampled_data.loc[sampled_data["Cluster_Labels"]!=0]

In [ ]:
#Demonstra a frequência dosclusters e cria um gráfico de barras que mostra a distribuição dos clusters
df_cluster_s_0.Cluster_Labels.value_counts().sort_values().plot(kind='bar', color=['green', 'lightgreen', 'yellow', 'purple'])

A análise que realizamos demonstra que existem muitos números zerados tanto em Carteira Industrial quanto em Carteira Agronegócio, sendo que a maior concentração de números zerados está contida na Carteira Agronegócio. Podemos verificar que existem mais Inadimplências em Agronegócio.

Além disso, verificamos que o cluster -1 tem maior concentração de outliers.

# Escolha do modelo

Ao analisar a separação dos clusters da carteira industrial entre os algoritmos, o modelo escolhido foi o K-Means, pelo fato da sua separação ter sido uma das mais lógicas em relação ao nível de inadimplência, apesar do modelo não classificar os fundos relacionando-os diretamente com as features. Logo, é possível concluir que nenhum dos modelos candidatos apresentou uma separação muito clara ou significativa entre os clusters. Enquanto o K-Means realizou uma separação em 3 partes proporcionais em relação ao target, o DBSCAN acabou formando um cluster muito grande por conta de uma densidade de dados muito alta em determina, resultando em uma divisão muito desigual e também pouco significativa.